In [ ]:
#module installations 
#uncomment the below if you are using mac
#!pip install Flask Flask-WTF werkzeug moviepy speechrecognition deep-translator gtts 

#uncomment the below if you are using windows
#pip install Flask Flask-WTF werkzeug moviepy speechrecognition deep-translator gtts




In [ ]:
# MINOR FIXES REQUIRED 
''' TODO
just improve user interface to the one i did yesterday

'''

import os
from flask import Flask, request, render_template_string, send_from_directory, redirect, url_for
from werkzeug.utils import secure_filename
import threading
from datetime import datetime
import moviepy.editor as mp
from moviepy.editor import TextClip, CompositeVideoClip, AudioFileClip
import speech_recognition as sr
from deep_translator import GoogleTranslator
from gtts import gTTS
from moviepy.audio.fx.all import audio_fadein, audio_loop


# Configuration
class Config:
    BASE_DIR = 'VideoTranslator'
    UPLOAD_FOLDER = os.path.join(BASE_DIR, 'uploads')
    PROCESSED_FOLDER = os.path.join(BASE_DIR, 'processed')
    ALLOWED_EXTENSIONS = {'mp4', 'avi', 'mov', 'mkv'}

# Create Flask app
app = Flask(__name__)
app.config.from_object(Config)

# Ensure the directories exist
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)
os.makedirs(app.config['PROCESSED_FOLDER'], exist_ok=True)

# Utility Functions and Classes
def allowed_file(filename):
    """Check if the file is an allowed video format."""
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in app.config['ALLOWED_EXTENSIONS']



class VideoProcessor:
    @staticmethod
    def extract_audio(video_path, audio_path):
        """Extract audio from the video file."""
        try:
            video = mp.VideoFileClip(video_path)
            video.audio.write_audiofile(audio_path)
        except OSError as e:
            raise OSError(f"Error extracting audio from video: {e}")
            

    @staticmethod
    def add_subtitles_and_audio(video_path, translated_text, translated_audio_path):
        """Add and sync translated audio with the video."""
        try:
            video = mp.VideoFileClip(video_path)

            # Load the translated audio
            translated_audio = AudioFileClip(translated_audio_path)

            # Ensure audio duration matches video duration
            if translated_audio.duration < video.duration:
                # Calculate the required playback speed to match video duration
                playback_speed = translated_audio.duration / video.duration
                slowed_audio = translated_audio.fx(mp.vfx.speedx, playback_speed)
            else:
                slowed_audio = translated_audio.set_duration(video.duration)

            # Set the audio for the video
            video_with_audio = video.set_audio(slowed_audio)
            
            # Set FPS for the video
            video_with_audio = video_with_audio.set_fps(24)

            return video_with_audio
        except Exception as e:
            raise Exception(f"Error adding audio to video: {e}")




class AudioProcessor:
    def __init__(self):
        self.recognizer = sr.Recognizer()

    def transcribe(self, audio_path, language='auto'):
        """Transcribe audio to text using Google Speech Recognition."""
        with sr.AudioFile(audio_path) as source:
            audio_data = self.recognizer.record(source)
            try:
                # Recognize speech using Google Speech Recognition
                text = self.recognizer.recognize_google(audio_data, language=language)
                return text
            except sr.UnknownValueError:
                print("Google Speech Recognition could not understand audio")
            except sr.RequestError as e:
                print(f"Could not request results from Google Speech Recognition service; {e}")
        return None

    def text_to_speech(self, text, output_audio_path, target_language='en', voice_type='male'):
        """Convert text to speech and save as an audio file."""
        tts = gTTS(text=text, lang=target_language, slow=False)
        tts.save(output_audio_path)

class Translator:
    @staticmethod
    def translate_text(text, target_language='es'):
        """Translate text to the target language using Google Translator."""
        translator = GoogleTranslator(source='auto', target=target_language)
        return translator.translate(text)



# Flask Routes and Handlers
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        file = request.files['file']
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            video_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(video_path)

            audio_path = os.path.join(app.config['UPLOAD_FOLDER'], 'audio.wav')
            target_language = request.form['language']
            voice_type = request.form['voice']
            date_str = datetime.now().strftime('%Y-%m-%d')
            output_filename = f"output_{filename.rsplit('.', 1)[0]}_{target_language}.mp4"
            processed_date_folder = os.path.join(app.config['PROCESSED_FOLDER'], date_str)
            os.makedirs(processed_date_folder, exist_ok=True)
            translated_audio_path = os.path.join(processed_date_folder, 'translated_audio.mp3')
            output_video_path = os.path.join(processed_date_folder, output_filename)

            # Process the video
            video_processor = VideoProcessor()
            audio_processor = AudioProcessor()
            translator = Translator()

            try:
                print("Extracting audio from video...")
                video_processor.extract_audio(video_path, audio_path)
                print("Audio extracted successfully.")
                
                print("Transcribing audio to text...")
                original_text = audio_processor.transcribe(audio_path)
                print(f"Original text: {original_text}")
                
                print("Translating text...")
                translated_text = translator.translate_text(original_text, target_language=target_language)
                print(f"Translated text: {translated_text}")
                
                print("Converting text to speech...")
                audio_processor.text_to_speech(translated_text, translated_audio_path, target_language=target_language)
                print("Text to speech conversion completed.")
                
                print("Adding subtitles and audio to video...")
                video_with_subtitles_and_audio = video_processor.add_subtitles_and_audio(video_path, translated_text, translated_audio_path)
                video_with_subtitles_and_audio.write_videofile(output_video_path, codec='libx264', audio_codec='aac')
                print("Video processing completed successfully.")
            except Exception as e:
                print(f"Error processing video: {e}")
                return f"Error processing video: {e}", 500

            return redirect(url_for('thank_you', date_str=date_str, filename=output_filename))

    return render_template_string('''
     <!doctype html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Video Translation and Dubbing</title>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
    <link href="https://fonts.googleapis.com/css2?family=Quicksand:wght@400;600&display=swap" rel="stylesheet">
    <style>
        body {
            padding-top: 50px;
            background-color: #f2f6fa;
            font-family: 'Quicksand', sans-serif;
        }
        .container {
            max-width: 600px;
            margin: auto;
            background: #ffffff;
            padding: 30px;
            border-radius: 10px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            transition: transform 0.3s ease, box-shadow 0.3s ease;
        }
        .container:hover {
            transform: translateY(-5px);
            box-shadow: 0 8px 12px rgba(0, 0, 0, 0.2);
        }
        h1 {
            margin-bottom: 20px;
            color: #004aad;
            font-weight: 600;
        }
        label {
            color: #004aad;
            font-weight: 600;
        }
        .btn-primary {
            background-color: #004aad;
            border-color: #004aad;
            transition: background-color 0.3s ease, transform 0.3s ease;
        }
        .btn-primary:hover {
            background-color: #00337a;
            transform: translateY(-2px);
        }
        .form-check-label {
            font-weight: 400;
        }
        .custom-file {
            position: relative;
            display: inline-block;
            width: 100%;
            height: calc(2.25rem + 2px);
            margin-bottom: 0;
        }
        .custom-file-input {
            position: relative;
            z-index: 2;
            width: 100%;
            height: calc(2.25rem + 2px);
            margin: 0;
            opacity: 0;
        }
        .custom-file-label {
            position: absolute;
            top: 0;
            right: 0;
            left: 0;
            z-index: 1;
            height: calc(2.25rem + 2px);
            padding: 0.375rem 0.75rem;
            font-weight: 400;
            line-height: 1.5;
            color: #495057;
            background-color: #fff;
            border: 1px solid #ced4da;
            border-radius: 0.25rem;
            transition: background-color 0.3s ease, border-color 0.3s ease, box-shadow 0.3s ease;
        }
        .custom-file-label::after {
            position: absolute;
            top: 0;
            right: 0;
            bottom: 0;
            z-index: 3;
            display: block;
            height: calc(2.25rem + 2px);
            padding: 0.375rem 0.75rem;
            line-height: 1.5;
            color: #495057;
            content: "Browse";
            background-color: #e9ecef;
            border-left: inherit;
            border-radius: 0 0.25rem 0.25rem 0;
        }
        
    </style>
</head>
<body>
    <div class="container">
        <h1 class="text-center">Video Translation and Dubbing</h1>
        <form method="post" enctype="multipart/form-data">
            <div class="form-group">
                <label for="file">Upload your video file:</label>
                <div class="custom-file">
                    <input type="file" class="custom-file-input" id="file" name="file" required>
                    <label class="custom-file-label" for="file">Choose file</label>
                </div>
            </div>
            <div class="form-group">
                <label for="language">Select target language:</label>
                <select class="form-control" id="language" name="language" required>
                <option value="es">Spanish</option>

<option value="fr">French</option>
<option value="de">German</option>
<option value="zh">Chinese</option>
<option value="ja">Japanese</option>
<option value="ru">Russian</option>
<option value="pt">Portuguese</option>
<option value="it">Italian</option>
<option value="ko">Korean</option>
<option value="ar">Arabic</option>
<option value="hi">Hindi</option>
<option value="bn">Bengali</option>
<option value="pa">Punjabi</option>
<option value="jv">Javanese</option>
<option value="mr">Marathi</option>
<option value="ta">Tamil</option>
<option value="te">Telugu</option>
<option value="kn">Kannada</option>
<option value="ml">Malayalam</option>
<option value="gu">Gujarati</option>
<option value="ur">Urdu</option>


                </select>
            </div>
            <div class="form-group">
                <label for="voice">Select voice type:</label>
                <select class="form-control" id="voice" name="voice" required>
                    <option value="male">Male</option>
                    <option value="female">Female</option>
                </select>
            </div>
            <button type="submit" class="btn btn-primary btn-block">Upload</button>
        </form>
    </div>
    <script>
        document.querySelector('.custom-file-input').addEventListener('change', function(e) {
            var fileName = document.getElementById("file").files[0].name;
            var nextSibling = e.target.nextElementSibling
            nextSibling.innerText = fileName
        })
    </script>
</body>
</html>
    
    ''')

@app.route('/thank_you/<date_str>/<filename>')
def thank_you(date_str, filename):
    return render_template_string('''
        <!doctype html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Thank You</title>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
    <style>
        body {
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            background-color: #f2f6fa;
        }
        .card {
            width: 100%;
            max-width: 600px;
            margin: auto;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            transition: transform 0.3s ease, box-shadow 0.3s ease;
        }
        .card:hover {
            transform: translateY(-5px);
            box-shadow: 0 8px 12px rgba(0, 0, 0, 0.2);
        }
        h1 {
            color: #004aad;
            font-weight: 600;
        }
        p {
            color: #333;
        }
        .btn-primary {
            background-color: #004aad;
            border-color: #004aad;
            transition: background-color 0.3s ease, transform 0.3s ease;
        }
        .btn-primary:hover {
            background-color: #00337a;
            transform: translateY(-2px);
        }
    </style>
</head>
<body>
    <div class="card text-center">
        <div class="card-body">
            <h1 class="card-title">Thank You!</h1>
            <p class="card-text">Your video has been successfully processed. You can view or download it using the links below.</p>
            <a href="{{ url_for('view_file', date_str=date_str, filename=filename) }}" class="btn btn-primary">View Online</a>
            <a href="{{ url_for('download_file', date_str=date_str, filename=filename) }}" class="btn btn-secondary">Download Video</a>
        </div>
    </div>
</body>
</html>
    ''', date_str=date_str, filename=filename)

@app.route('/view/<date_str>/<filename>')
def view_file(date_str, filename):
    return render_template_string('''
        <!doctype html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>View Video</title>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
    <style>
        body {
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            background-color: #f2f6fa;
        }
        .card {
            width: 100%;
            max-width: 800px;
            margin: auto;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            transition: transform 0.3s ease, box-shadow 0.3s ease;
        }
        .card:hover {
            transform: translateY(-5px);
            box-shadow: 0 8px 12px rgba(0, 0, 0, 0.2);
        }
        h1 {
            color: #004aad;
            font-weight: 600;
        }
        video {
            width: 100%;
            border-radius: 10px;
        }
    </style>
</head>
<body>
    <div class="card text-center">
        <div class="card-body">
            <h1 class="card-title">Your Processed Video</h1>
            <video controls>
                <source src="{{ url_for('download_file', date_str=date_str, filename=filename) }}" type="video/mp4">
                Your browser does not support the video tag.
            </video>
        </div>
    </div>
</body>
</html>
    ''', date_str=date_str, filename=filename)


@app.route('/downloads/<date_str>/<filename>')
def download_file(date_str, filename):
    directory = os.path.join(app.config['PROCESSED_FOLDER'], date_str)
    return send_from_directory(directory, filename, as_attachment=True)


def run_flask_app():
    app.run(port=5004, debug=True, use_reloader=False)

if __name__ == '__main__':
    # Start Flask app in a separate thread
    flask_thread = threading.Thread(target=run_flask_app)
    flask_thread.start()
